In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']


In [3]:
import os
import sys
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import googletrans
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [4]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [5]:
df_jobs = df_jobs.sample(n=100, random_state=1)

In [6]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [7]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 15019 to 20563
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     100 non-null    object 
 1   Platform           100 non-null    object 
 2   Job ID             100 non-null    object 
 3   Job Title          100 non-null    object 
 4   Company Name       100 non-null    object 
 5   Location           100 non-null    object 
 6   Job Description    100 non-null    object 
 7   Rating             5 non-null      float64
 8   Employment Type    99 non-null     object 
 9   Company URL        96 non-null     object 
 10  Job URL            100 non-null    object 
 11  Job Age            100 non-null    object 
 12  Job Age Number     100 non-null    object 
 13  Collection Date    100 non-null    object 
 14  Data Row           95 non-null     float64
 15  Tracking ID        95 non-null     object 
 16  Industry           9

In [8]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [9]:
job_descriptions = list(set(df_jobs['Job Description'].to_list()))


In [10]:
len(job_descriptions)


100

In [11]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize
nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
nltk_sentencizer = []

for job_description in job_descriptions:
    doc = sent_tokenize(job_description)
    nltk_sentencizer.extend(doc)

In [29]:
len(nltk_sentencizer)

145

In [14]:
nltk_sentencizer[3]

'At Uber, we ignite opportunity by setting the world in motion We take on big problems to help drivers, riders, delivery partners, and eaters get moving in more than 10,000 cities around the world, , We welcome people from all backgrounds who seek the opportunity to help build a future where everyone and everything can move independently If you have the curiosity, passion, and collaborative spirit, work with us, and lets move the world forward, together, , , , As a member of the team, you will work alongside Community Operations Managers to improve our customers support experience You will support strategic decisionmaking and process improvement through datadriven problem solving and collaboration across key Community Operations functions eg Agent Enablement, Analytics  Insights, Program Management, Strategy  Planning, , For this role you should have high attention to detail, strong verbal and written communication skills, excellent organization and time management, and a bias towards 

In [22]:
# Set up Spacy
import spacy
from spacy.lang.en import English

try:
    nlp = English()
except:
    nlp = spacy.load('en_core_web_sm')

custom_punct_chars = ['\n', '\n\n', '\n\n\n', ',,', ', ,', ', , ', ',,,', ', , ,', ', , , ', ',,,,', ', , , ,', ', , , , ', ',,,,,', ', , , , ,', ', , , , , ', ':', '|']


In [23]:
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)


In [24]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

In [25]:
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    new_punct_chars = pickle.load(f)

In [26]:
spacy_sentencizer = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
        doc = nlp(job_description)
        spacy_sentencizer.extend([sent.text for sent in doc.sents])


In [27]:
len(spacy_sentencizer)

224

In [28]:
spacy_sentencizer[3]

'At Uber, we ignite opportunity by setting the world in motion We take on big problems to help drivers, riders, delivery partners, and eaters get moving in more than 10,000 cities around the world, , We welcome people from all backgrounds who seek the opportunity to help build a future where everyone and everything can move independently If you have the curiosity, passion, and collaborative spirit, work with us, and lets move the world forward, together, , , , As a member of the team, you will work alongside Community Operations Managers to improve our customers support experience You will support strategic decisionmaking and process improvement through datadriven problem solving and collaboration across key Community Operations functions eg Agent Enablement, Analytics  Insights, Program Management, Strategy  Planning, , For this role you should have high attention to detail, strong verbal and written communication skills, excellent organization and time management, and a bias towards 

In [ ]:
tokenizer = nlp.tokenizer

spacy_tokenizer = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
        doc = tokenizer(job_description)
        spacy_tokenizer.extend([token.text for token in doc])


In [ ]:
len(spacy_tokenizer)

In [ ]:
spacy_tokenizer[:10]

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
for sentence in spacy_sentencizer:
    print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)